In [15]:
from wallcheb.qtmlib.circuits.lcu import LCUMultiplexorBox
from guppylang import guppy
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array, h
from guppylang.std.builtins import array, exit
from typing import Callable
from guppylang.std.debug import state_result


In [16]:
from wallcheb.operators import generate_pytket_hvs_hubbard

u = 1
n_sites = 2
m = 3
product_block_encoding_qpo, hubbard_hamiltonian = generate_pytket_hvs_hubbard(u, n_sites, m)
n_state_qubits = 2*n_sites


/Users/nathanfitzpatric/Github/wall_chebyshev/wallcheb/operators/hubbard_model.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[i,j] = hmat[subset[i], subset[j]]


In [17]:
from numpy.linalg import eigh

e,c = eigh(hubbard_hamiltonian)
c[:,0], e[0]

(matrix([[-0.00000000e+00-0.j],
         [-0.00000000e+00-0.j],
         [-2.22044605e-16+0.j],
         [-4.35162146e-01+0.j],
         [ 0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j],
         [ 5.57345410e-01+0.j],
         [ 0.00000000e+00+0.j],
         [-9.49975516e-17+0.j],
         [-5.57345410e-01+0.j],
         [ 0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j],
         [-4.35162146e-01+0.j],
         [-0.00000000e+00+0.j],
         [-0.00000000e+00+0.j],
         [-0.00000000e+00-0.j]]),
 np.float64(-1.5615528128088303))

In [18]:
lcu_box = LCUMultiplexorBox(product_block_encoding_qpo[0], n_state_qubits)
n_prep_qubits = lcu_box.n_prepare_qubits

In [19]:
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes


def build_multiplexor_lcu(ham, n_state_qubits, ind):
    multiplexor_lcu = LCUMultiplexorBox(ham, n_state_qubits)
    

    circ = multiplexor_lcu.get_circuit()
    DecomposeBoxes().apply(circ)
    rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
    rebase.apply(circ)


    qlibs_multiplexor_lcu = guppy.load_pytket(f"qlibs_multiplexor_lcu_{ind}", circ)
    return qlibs_multiplexor_lcu

In [20]:
@guppy.comptime
def guppy_prod_circs() -> array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]],None], comptime(m)]:

    guppy_circuits = [build_multiplexor_lcu(qpo, n_state_qubits, i) for i, qpo in enumerate(product_block_encoding_qpo)]
    return guppy_circuits

In [21]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, discard_array, measure, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from typing import Callable

@guppy
def product_block_encoding(prod_block_encoding: array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]], None], comptime(m)], state_qreg: array[qubit, comptime(n_state_qubits)]) -> None:
    

    for prod_block in prod_block_encoding.copy():

        prep_qreg = array(qubit() for _ in range(comptime(n_prep_qubits)))
        prod_block(prep_qreg, state_qreg)

        outcome = measure_array(prep_qreg)
        # result("c", measure_array(prep_qubits))
        for b in outcome: 
            if b:
                exit("circuit failed",1)




In [22]:
@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    state_qreg = array(qubit() for _ in range(comptime(n_state_qubits)))
    
    for i in range(comptime(n_state_qubits)):
        h(state_qreg[i])

    prods = guppy_prod_circs()

    product_block_encoding(prods, state_qreg)

    state_result("gs", state_qreg)
    # result('c',measure_array(state_qreg))
    discard_array(state_qreg)


compiled_hugr = guppy.compile(main)


In [23]:
from hugr.qsystem.result import QsysResult
from selene_sim import build, Quest

n_shots = 1000
runner = build(compiled_hugr)
shots = QsysResult(
    runner.run_shots(
        Quest(), n_qubits=lcu_box.n_qubits, n_shots=n_shots
    )
)

# print(f'Final state:{shots.register_counts()["c"]}, when initial state is |000>')

for shot in shots.results:
    print(shot)

states = Quest.extract_states_dict(shots.results[0].entries)
print(states)
state_vector1 = states["gs"].get_single_state()
print(state_vector1)

# shots_counts = shots.register_counts()["c"]
# success_prob = sum(shots_counts.values())/ n_shots
# print(f"Success probability (from circuits): {success_prob:.4f}")

QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed', 1)])
QsysShot(entries=[('exit: circuit failed

KeyError: 'gs'